# A simple matrix product state (MPS) emulator

In this notebook, we show a very simple implementation of a MPS emulator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tools import Circuit, gate_dic

class MPSState:
    """class containing the MPS representation of a quantum state
    
    the MPS is stored as a list of tensors.
    
    Attributes:
        nqbits (int): number of qubits
        tensors (list<np.array>): list of tensors
        bond_dim (int): bond dimension
        
    Args:
        nqbits (int): the number of qubits
        bond_dim (int): the bond dimension (chi)
    """
    
    def __init__(self, nqbits, bond_dim=2):
        """initialize to |0, 0, ...0>
        

        """
        self.nqbits = nqbits
        self.tensors = [np.zeros((1, 2, 1), np.complex128) for _ in range(nqbits)]
        # state |0, 0, ..., 0>
        for qb in range(nqbits):
            self.tensors[qb][0, 0, 0] = 1.0
        self.bond_dim = bond_dim
        
    def apply(self, gate, qbits, param=None):
        """
        Apply a gate to the current state.
        
        Args:
            gate (string): name of gate
            qbits (list<int>): qubits on which it acts
            param (float, optional): parameter of gate (if applicable)
        
        """
        # ...
        if len(qbits) > 2: raise Exception(f"The gate acts on too many ({len(qbits)}) qubits")
        gate_ = gate_dic[gate] if param is None else gate_dic[gate](param)
        # gate_ is a (2, 2) or (4,4) array
        if len(qbits) == 1:
            # apply the gate
            # Hint: use tensordot function
            self.tensors[qbits[0]] = # ...
        else:
            # put qubits in right order
            if qbits[0] < qbits[1]:
                q1, q2 = qbits[0], qbits[1] 
            else:
                q1, q2 = qbits[1], qbits[0]
                gate_ = np.reshape(gate_, (2, 2, 2, 2))
                # swap q1 and q2
                gate_ = np.transpose(gate_, (1, 0, 3, 2))
                gate_ = np.reshape(gate_, (4, 4))
                
            if q1 != q2 - 1: 
                raise Exception("the gate must be applied on neighboring qubits")
                
            # merge the two neighboring tensors
            # Hint: tensordot
            
            # apply the gate
            # Hint: tensordot
            
            # now compress by SVD
            # ...
            
            # renormalize so that sum s^2 = 1
            # ...
            
            # discard unnecessary columns/rows of u and vh
            # ...
            
            self.tensors[q1] = # ...
            self.tensors[q2] = # ...
            
        
    def __str__(self):
        string = ""
        for qb in range(self.nqbits):
            string += str(self.tensors[qb].shape) + "\n"
        return string
        
        
    def to_vec(self):
        """contract to dense vector"""
        tmp = self.tensors[0]
        for qb in range(1, self.nqbits):
            tmp = np.tensordot(tmp, self.tensors[qb], axes=1)
        return np.reshape(tmp, 2**self.nqbits)
    
class MPSQPU:
    "Simulator based on MPS"
    def __init__(self, nqbits, bond_dim=2):
        self.state = MPSState(nqbits, bond_dim)
        
    def submit(self, circuit):
        """
        Args:
            circuit (Circuit): a quantum circuit to be simulated
        """
        assert(circuit.nqbits == self.state.nqbits)
        for gate_tuple in circuit.gates:
            self.state.apply(*gate_tuple)
            
        return self.state

# a couple of tests
psi = MPSState(2)
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 0, 0]))<1e-13)

psi.apply("H", [0])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 1, 0])/np.sqrt(2))<1e-13)

psi.apply("CNOT", [0, 1])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 0, 1])/np.sqrt(2))<1e-13)

psi.apply("CNOT", [0, 1])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 1, 0])/np.sqrt(2))<1e-13)

psi.apply("CNOT", [1, 0])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 1, 0])/np.sqrt(2))<1e-13)

psi.apply("RY", [1], np.pi/2)
assert(np.linalg.norm(psi.to_vec() - np.array([1, 1, 1, 1])/2)<1e-13)

In [ ]:
# test with a circuit
circ = Circuit(2, [("H", [0]), ("CNOT", [0, 1])])

qpu = MPSQPU(2)
res = qpu.submit(circ)

print(res.to_vec()) # what do you expect?

## With random circuits

In [ ]:
from tools import random_circuit
from state_vector_qpu import StateVectorQPU # a Schrödinger-style dense representation simulator
nqbits = 10

circ = random_circuit(nqbits, 6)
qpu = StateVectorQPU(nqbits, gate_dic)
res = qpu.submit(circ)

bond_dim_list = [2, 4, 8, 16, 32, 64]

fid_list = []
for bond_dim in bond_dim_list:
    qpu = MPSQPU(nqbits, bond_dim=bond_dim)
    res_mps = qpu.submit(circ)

    fid = abs(np.dot(np.conj(res.to_vec()), res_mps.to_vec()))**2
    print(bond_dim, fid)
    fid_list.append(fid)
    
plt.plot(bond_dim_list, fid_list, '-o', lw=3)
plt.xlabel("Bond dimension")
plt.ylabel("Fidelity")
plt.grid();

## Going further

- compute the run time as a function of the bond dimension
- implement the measurement of an observable $H$ decomposed on the Pauli basis, $H = \sum_i \lambda_i P_i$.
- implement the canonicalization of the MPS
- run the emulator on circuits with 100 qubits!